In [1]:
import h2o
from h2o.automl import H2OAutoML
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from score import rmsle

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM JBR-11.0.13.7-1751.21-jcef (build 11.0.13+7-b1751.21, mixed mode)
  Starting server from C:\Users\henri\anaconda3\envs\tdt4173\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\henri\AppData\Local\Temp\tmpn2z7v5bu
  JVM stdout: C:\Users\henri\AppData\Local\Temp\tmpn2z7v5bu\h2o_henri_started_from_python.out
  JVM stderr: C:\Users\henri\AppData\Local\Temp\tmpn2z7v5bu\h2o_henri_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Europe/Berlin
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.4
H2O_cluster_version_age:,"2 years, 4 months and 9 days !!!"
H2O_cluster_name:,H2O_from_python_henri_w2kigl
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.480 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"accepting new members, healthy"


In [30]:
df = h2o.import_file("../../own_data/all_merged_h2o.csv")
train = h2o.import_file("../../own_data/train_log_h2o.csv")
test = h2o.import_file("../../own_data/test_h2o.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [31]:
pd_df = pd.read_csv("../../own_data/all_merged_h2o.csv")
categorical_features = list(pd_df.select_dtypes(include=[np.object0]).columns)

In [32]:
df[categorical_features] = df[categorical_features].asfactor()
train[categorical_features] = train[categorical_features].asfactor()
test[categorical_features] = test[categorical_features].asfactor()

In [33]:
y = "revenue"
x = df.columns
x.remove(y)

In [34]:
aml = H2OAutoML(max_models=2, seed=1)
aml.train(x = x, y = y, training_frame=train)

AutoML progress: |
19:18:22.276: AutoML: XGBoost is not available; skipping it.

████████████████████████████████████████████████████████| 100%


In [35]:
lb = aml.leaderboard

In [36]:
lb.head()

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_BestOfFamily_AutoML_20221010_191822,0.513004,0.716243,0.513004,0.540392,0.314056
StackedEnsemble_AllModels_AutoML_20221010_191822,0.513004,0.716243,0.513004,0.540392,0.314056
DRF_1_AutoML_20221010_191822,0.521573,0.7222,0.521573,0.54193,0.314854
GLM_1_AutoML_20221010_191822,0.563083,0.750389,0.563083,0.579573,0.32544


In [57]:
preds = aml.predict(test)
preds = preds.as_data_frame()

stackedensemble prediction progress: |████████████████████████████████████| 100%


c:\Users\henri\anaconda3\envs\tdt4173\lib\site-packages\h2o\job.py:70: UserWarning: Test/Validation dataset column 'store_id' has levels not trained on: [810930632-971672927-6658, 811583332-993885584-487794, 811608262-911610922-433688, 811641472-911653656-475280, 811721182-911750473-475583, 811743402-997035518-167920, 811751332-911785153-542454, 811770892-812013432-623777, 811823112-996654931-149641, 811897272-974083299-48251, 812027212-912082512-626558, 812099752-997671414-168018, 812171062-912205908-629415, 812198262-912238768-630679, 812320262-912368319-638028, 812418572-990541035-12837, 812445952-812498762-643432, 812546252-984864086-6441, 812547992-986130152-63117, 812637932-985209081-8717, 812658042-912666042-649264, 812720112-912739082-651825, 812786822-972451185-254380, 812864882-812886592-658026, 813059142-913108906-670038, 813159252-888514082-45852, 813221322-913237803-676724, 813326892-913387422-682558, 813347172-813390302-682542, 813390132-913491823-686519, 813413892-981005

In [58]:
preds = np.array(preds["predict"])
preds = np.expm1(preds)

In [60]:
true_revenue = test.as_data_frame().revenue
rmsle(true_revenue, preds)

0.7154075054814424

In [61]:
test["revenue"]

revenue
6.853
1.733
3.721
50.238
0.26
3.169
16.69
5.417
3.349
4.389
